## M2.5 - Creating a Reproducible Climate Data Analysis

*Part of:* [**Computational Climate Science**](https://github.com/OpenClimateScience/M2-Computational-Climate-Science) | **Previous Lesson** | **Next Lesson**

**Contents:**

- [Creating a reproducible workflow](#Creating-a-reproducible-workflow)
  - [Our workflow: Downloading the data (Step 1)](#Our-workflow:-Downloading-the-data-(Step-1))
  - [Our workflow: Data processing (Step 2)](#Our-workflow:-Data-processing-(Step-2))
- [A reproducible project's files](#A-reproducible-project's-files)
- [Comparing multiple years of climate data](#Comparing-multiple-years-of-climate-data)
  - [Computing TOA radiation](#Computing-TOA-radiation)
  - [Computing PET](#Computing-PET)
  - [Resampling the CHIRPS data](#Resampling-the-CHIRPS-data)
  - [Selecting part of an xarray time series](#Selecting-part-of-an-xarray-time-series)

---

## Creating a reproducible workflow

In the previous lesson, we used `dask` and `xarray` to read a collection of netCDF files, then **mapped** a **vectorized function** over each array in a time series. We produced a graph that showed the Precipitation-to-PET ratio for Tiaret, Algeria, in the early months of 2024, during a severe drought.

On its own, the graph doesn't tell us how severe the drought in Tiaret is. Although precipitation in the region has replenished less than 5% of its lost water over the past few months, this could just be part of the normal seasonal cycle. Actually, we know that January through April is a relatively wet period for Tiaret, but the question remains: **Can we compare this year to past years?**

Whenever we want to apply a completed analysis to a new dataset, either over time or space, that's an opportunity for us to improve how our workflow is represented. Consider the two scripts below, which represent the first two steps in our workflow.

### Our workflow: Downloading the data (Step 1)

The first file might be named something like **`YYYYMMDD_step1_download_MERRA2_data.py`**. Remember that `YYYYMMDD` is today's date, and it will help us to **link our output files with the code that created them.**

Note that Python files, with the `*.py` file extension, can have a **file-level docstring,** which in the example below is the Python multi-line string beginning with `'''`. File-level docstrings must begin on the first line of a file. They are extremely important for reproducible workflows; the first line of a file is the first place you'll look to understand what the purpose of the file is!

```python
'''
Downloads MERRA-2 M2SDNXSLV data, for the first 5 months of a year.
Read more about MERRA-2 here:

    https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/

Data are downloaded to this folder:

    data_raw/MERRA2
'''

import earthaccess

DATA_YEAR = 2023

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = (f"{DATA_YEAR}-01-01", f"{DATA_YEAR}-05-31"))

# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')
```

&#x1F449; **From top to bottom, note that:**

- We have a **file-level docstring** at the top of script with important information about the purpose of the script, where to get more information, and how the script changes our file system.
- All of our `import` statements are near the top of the script. This signals to someone reading our script what Python modules are required to run the script. We don't want to put any `import` statements farther down in the script because it would be harder to find them. This could lead to a surprise `ImportError` when running the script.
- Parameters that might be changed when running the script are clearly identified, using all capital letters to define the variable, near the top of the script. For example, `DATA_YEAR` is a variable we might want to change when running the script multiple times. Putting it at the top of our script, using all capital letters, helps avoid the difficulty of reading through every line of the script to find the part that needs to change.

### Our workflow: Data processing (Step 2)

The next step is to read-in the data files and calculate top-of-atmosphere (TOA) radiation. The second file might be named **`YYYYMMDD_step2_compute_TOA_radiation.py`**.

```python
'''
Computes top-of-atmosphere (TOA) radiation from a series of MERRA-2 
M2SDNXSLV files, then writes an output netCDF file. TOA radiation is
calculated according to the FAO formula for extraterrestrial radiation:

    https://www.fao.org/4/X0490E/x0490e07.htm#radiation
'''

import numpy as np
import xarray as xr

# NOTE: This will be different on your computer system and you should
#   use an absolute path, not a relative path
MERRA2_DATA_DIR = './data_raw/MERRA2'
DATA_YEAR = 2023
OUTPUT_FILE = f'./outputs/YYYYMMDD_MERRA2_{DATA_YEAR}_with_TOA-radiation.nc'

def main():
    ds = xr.open_mfdataset(f'{MERRA2_DATA_DIR}/*{DATA_YEAR}*.nc4', chunks = 'auto')
    lats = ds['lat'].values.reshape((361, 1, 1))\
        .repeat(ds.lon.size, axis = 1)\
        .repeat(ds.time.size, axis = 2)
    ds['lat_grid'] = (('lat', 'lon', 'time'), lats)

    # Compute TOA radiation
    template = ds['T2MMEAN']
    template.name = 'toa_radiation'
    result = xr.map_blocks(toa_radiation_wrapper, ds, template = template)
    toa_result = result.compute()
    # Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
    ds['toa_radiation'] = toa_result * 0.408
    
    # Write the file to disk, just the important variables
    ds = ds[['T2MMAX', 'T2MMEAN', 'T2MMIN', 'toa_radiation']]
    comp = dict(zlib = True, complevel = 5)
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(OUTPUT_FILE, format = 'NETCDF4', encoding = encoding)

    
def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the earth-sun
    distance; w is the sunset hour angle; and D is the solar declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    
    # Convert latitude from degrees to radians
    lat_radians = np.deg2rad(latitude)
    # Earth-Sun distance, as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos(doy * ((2 * pi) / 365))
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(doy * ((2 * pi) / 365) - 1.39)
    
    # Sunset hour angle; we use np.where() below to guard against
    #   warnings where arccos() would return invalid values, which
    #   happens when the argument is outside [-1, 1]
    _hour_angle = -np.tan(lat_radians) * np.tan(declination)
    _hour_angle = np.where(np.abs(_hour_angle) > 1, np.nan, _hour_angle)
    sunset_hour_angle = np.arccos(_hour_angle)
    
    return ((24 * 60) / pi) * solar_constant * earth_sun_dist *\
        (sunset_hour_angle * np.sin(lat_radians) * np.sin(declination) +
            np.cos(lat_radians) * np.cos(declination) * np.sin(sunset_hour_angle))


def toa_radiation_wrapper(dataset):
    '''
    Wraps toa_radiation to work with an xarray.Dataset

    Parameters
    ----------
    dataset : xarray.Dataset
        Input dataset with "lat_grid" and "time" variables

    Returns
    -------
    xarray.DataArray
    '''
    return toa_radiation(dataset['lat_grid'], dataset['time.dayofyear'])


# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

&#x1F449; **Again, note that:**

- A **file-level docstring** and `import` statements near the top of the script helps users identify the purpose of the script and what Python modules are required to run it.
- Our `toa_radiation()` function also has a **function-level docstring** that describes how the function works: what **input arguments** it requires and what the **return value** is.

&#x1F449; **Consider the line that contains `if __name__ == '__main__'`; what does this mean?**

- Every Python file, or `*.py` file, has code that can be executed in two ways, either by running `python myscript.py` (as a script) or by *importing* the file as a module, e.g., `import myscript`.
- Every Python file, when the code it contains is run, introduces a variable called `__name__` that indicates the name of the module. When a Python file is executed as a script, instead, then `__name__` is set equal to `'__main__'`. Therefore, `__name__` is a variable that we can use to test whether or not the Python code is currently being run as a script or if it was imported as a module.

**Why do we care about whether the file is being run as a script or if it was imported as a module?** When a Python file is imported as a module, all of the code in that file is executed. This means that any Python code that is outside of a **function definition** will be executed every time we import the module, which is probably not what we want, especially if the file contains useful functions like `toa_radiation()` that we might want to **re-use** elsewhere; that is, we might want to write something like `from myscript import toa_radiation()` in another script.

The code that we want to execute *only when the file is run as a script* should be placed in a function like `main()`, which can be called conditionally:
```python
# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

If this is confusing then, for now, just consider the `if` statement above to be a "magic" Python technique that allows us to write Python code files that can both be executed as scripts and imported as modules.

---

## A reproducible project's files

There are other steps in our analysis, but we leave it up to you as an exercise to write re-useable, well-documented scripts like the one above. Your final project directory might look something like this:

![](./assets/M2_file_tree_workflow.png)

&#x1F449; **Note that each of the `scripts` files includes the words `step1`, `step2`, or `step3`, indicating the order in which the workflow should be exected.** This will someone figure out in what order the workflow should be executed; it might even help you if you come back to this project months later and can't remember what you've done previously!

---

## Comparing multiple years of climate data

Ultimately, we do want to answer our question from the last lesson: How does the precipitation-to-PET ratio for Winter 2024 compare to that of Winter 2023 in Tiaret?

Our reproducible workflow, above, could be used to answer this question. Let's also see how to answer the question in an **interactive Python session,** i.e., this Jupyter Notebook. If we've already downloaded the MERRA-2 data for both 2023 and 2024, we're ready to load the data using `xarray.open_mfdataset()`.

In [ ]:
import xarray as xr
from matplotlib import pyplot

ds_chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')
ds_merra2 = xr.open_mfdataset('data_raw/MERRA2/*.nc4', chunks = 'auto')

Our Python script, `YYYYMMDD_step2_compute_TOA_radiation.py`, contains a useful, reusable function, `toa_radiation()`. **How can we make use of that function without copying and pasting the function declaration into another script, or into this Notebook?** In general, we should avoid having more than one copy of any function. One of the main reasons to write a function is so that we don't have to write the same code twice!

&#x1F449; As suggested above, when we included the `if __name__ == '__main__':` statement in our script, detecting the module's name with `__name__` is a useful way to figure out whether Python code is being executed as a script or as a module. As long as we can see the script named `YYYYMMDD_step2_compute_TOA_radiation.py` somewhere in our file tree, then we should be able to import it as a module. Below, we use the `os` library to look at the current directory's files. We can see that there is a `scripts` directory below this one...

In [ ]:
import os

# Display the files in the current directory
files = os.listdir('.') # The single dot "." indicates the current directory
files.sort()
files

And inside `scripts` is the Python script we want to import as a module!

In [ ]:
os.listdir('scripts')

How do we do that? Because both directories and Python `*.py` files can be treated like they are modules, all we need to do is write a module `import` statement like the one below.

In [ ]:
from scripts.YYYYMMDD_step2_compute_TOA_radiation import toa_radiation_wrapper

Note that we just imported one variable name from this script, `toa_radiation_wrapper()` function, which gives us access to the `toa_radiation()` function. And because we wrote such a helpful **function-level docstring** earlier, we can see all the information on how to use this function by calling `help()` on our function.

In [ ]:
help(toa_radiation_wrapper)

### Computing TOA radiation

As before, in order to compute the TOA radiation, we want to create a latitude grid so that we can **vectorize** our `toa_radiation()` step.

In [ ]:
lat_grid = ds_merra2.lat.values.reshape(1, 361, 1)\
    .repeat(ds_merra2.lon.size, axis = 2)\
    .repeat(ds_merra2.time.size, axis = 0)
lat_grid.shape

In [ ]:
ds_merra2['lat_grid'] = (('time', 'lat', 'lon'), lat_grid)
ds_merra2

And we need the day of year, but that is already available as `ds_merra2['time.dayofyear']`.

Therefore, we're ready to calculate TOA radiation!

In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
toa_rad = toa_radiation_wrapper(ds_merra2) * 0.408
toa_rad.sel(time = '2024-05-01').plot()

### Computing PET

And we'll go ahead and subset the MERRA-2 data to Tiaret also.

In [ ]:
toa_rad.shape

In [ ]:
import numpy as np

ds_merra2['toa_radiation'] = toa_rad

# Select just the Tiaret region
merra2_tiaret = ds_merra2.sel(lon = -1.32, lat = 35.37, method = 'nearest')

# Compute PET using the Hargreaves equation
merra2_tiaret_pet = 0.0023 * merra2_tiaret['toa_radiation'] * np.sqrt(
    merra2_tiaret['T2MMAX'] - merra2_tiaret['T2MMIN']) * (merra2_tiaret['T2MMEAN'] + 17.8)

### Resampling the CHIRPS data

Finally, we need to process the CHIRPS monthly precipitation data into daily precipitation, for the Tiaret area, as we did in the last lesson.

In [ ]:
chirps_tiaret = ds_chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret_daily = chirps_tiaret.resample(time = 'D').nearest().mean(['x', 'y']) / 30
chirps_tiaret_daily

### Selecting part of an `xarray` time series

Now we're ready to compare 2023 and 2024.

Assuming that our `data_raw/MERRA2` directory contains the data we downloaded from both 2023 and 2024, then our `ds_merra2` dataset contains data from both 2023 and 2024. We saw how to select a specific date using the `sel(time = 'YYYY-MM-DD')` pattern, but how can we select all the data from a specific year, month, or range of dates?

Every `xarray` dimension that has the `datetime64[ns]` type also a `DatetimeAccessor`, accessed through the `dt` property.

In [ ]:
merra2_tiaret_pet.time.dt

[The `dt` property can be used to subset a time series.](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html)

In [ ]:
merra2_tiaret_pet.time.dt.year

For example, to select only the data from 2023...

In [ ]:
# Select only those data points along the time access where time.year is in a list of years
merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([2023]))

In [ ]:
ratios = []
precip = []
for year in [2023, 2024]:
    pet_this_year = merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([year]))
    precip_this_year = chirps_tiaret_daily.sel(time = chirps_tiaret_daily.time.dt.year.isin([year]))
    # Select the first 122 days
    pet_this_year = pet_this_year.isel(time = slice(0, 122))
    precip_this_year = precip_this_year.isel(time = slice(0, 122))
    # NOTE: We must use the .values attribute because these are from two different datasets
    ratios.append(100 * precip_this_year.values / pet_this_year.values)
    precip.append(precip_this_year.values)

In [ ]:
pyplot.figure(figsize = (12, 5))
for y, year in enumerate([2023, 2024]):
    pyplot.plot(pet_this_year.time, ratios[y], label = year)
    
pyplot.ylabel('Precipitation-to-PET Ratio (mm day-1)')
pyplot.legend()

**The plot above suggests that the hydrologic conditions in early 2024 aren't that different from the previous year.** Why did these seasonally dry conditions produce a **socioeconomic drought** in 2024 but not in 2023? As we saw when we plotted the long-term precipitation-PET ratios, using TerraClimate data, wet-season precipitation has been lower than usual for several years. One explanation for the 2024 socioeconomic drought could be that the *cumulative* effect of several years of low precipitation have created drought conditions in 2024 that were not as apparent in 2023.

We can visualize the long-term decline in winter precipitation again using the CHIRPS data. Despite a relative uptick in precipitation in 2022, it's apparent that winter precipitation has been unusually low since 2019.

In [ ]:
chirps_tiaret.sel(time = chirps_tiaret.time.dt.month.isin([1,2,3,4,5])).groupby('time.year').sum().mean(['x', 'y']).plot()
pyplot.ylabel('January - May Precipitation (mm)')

---

## More resources

- [`xarray.core.accessor_dt.DatetimeAccessor` (Documentation)](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html)